In [1]:
import os
import pandas as pd
import joblib
import numpy as np

from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

/opt/anaconda/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_pickle('./assets/clean-data/data_set_master.pickle')

In [3]:
df = pd.read_pickle('./assets/clean-data/full_data_161718_bucketed.pickle')

print(len(df))
print(df.head())

# train, valid = train_test_split(df, test_size=0.05)
train = joblib.load('train-fare.joblib')

bdt_real = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    n_estimators=200,
    learning_rate=1)

df = df[['PULocationID', 'DOLocationID', 'day_of_week', 't_bucket',
              'month', 'cluster_id', 'fare_bucket', 'dist_bucket']]

df = df.drop(['month'], axis=1)

xTrain = train.drop(['DOLocationID', 'dist_bucket', 'fare_bucket'], axis=1)
yTrain = train['fare_bucket']

bdt_real.fit(xTrain, yTrain)

joblib.dump(bdt_real, './ada-models/ada-200-fare-2-wclus.joblib')

9657394
   cluster_id  day_of_week  fare_amount  month  t_bucket  trip_distance  \
0         421            0          8.5      7        15            1.4   
1        2316            3          4.0      7        28            0.5   
2         399            3         11.0      7         5            2.7   
3        1480            3          5.5      7        18            0.6   
4        1393            3          6.5      7        23            0.9   

   fare_bucket  dist_bucket  
0            2            2  
1            1            0  
2            3            4  
3            2            1  
4            2            1  


In [ ]:
# df = pd.read_pickle('./assets/data_set_master.pickle')
# train, valid = train_test_split(df, test_size=0.05)

classifier = joblib.load('./ada-models/ada-200-fare-2-wclus.joblib')

valid = joblib.load('valid-fare.joblib')

xTest = valid.drop(['DOLocationID', 'dist_bucket', 'fare_bucket'], axis=1)
yTest = valid['fare_bucket']

predictions = classifier.predict(xTest)

valid['predictions'] = predictions
valid.to_pickle('./assets/predictions_fare.pickle')

print(np.mean(yTest == predictions))
print(classifier.feature_importances_)

In [ ]:
df = pd.read_pickle('./assets/activity_master.pickle')

print(len(df))
print(df.head())

train, valid = train_test_split(df, test_size=0.05)
# train = joblib.load('train_fare.joblib')

bdt_real = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=3),
    n_estimators=200,
    learning_rate=1)

xTrain = train.drop(['activity', 'activity_bucket'], axis=1)
yTrain = train['activity_bucket']

bdt_real.fit(xTrain, yTrain)

joblib.dump(bdt_real, './ada-models/ada-200-act-3.joblib')
joblib.dump(train, 'train-activity.joblib')
joblib.dump(valid, 'valid-activity.joblib')

In [ ]:
classifier = joblib.load('./ada-models/ada-200-act-3.joblib')

valid = joblib.load('valid-activity.joblib')
xTest = valid.drop(['activity', 'activity_bucket'], axis=1)
yTest = valid['activity_bucket']

predictions = classifier.predict(xTest)

valid['predictions'] = predictions
valid.to_pickle('./assets/predictions_fare.pickle')

print(np.mean(yTest == predictions))
print(classifier.feature_importances_)